In [1]:
import torch
from tifffile import imsave

from config import Eval
from model import UNet2D
from data import getDataLoader

# config
config = Eval()
device = torch.device('cuda')

# model
net = UNet2D(config).to(device)  # config not important since load
net.load_state_dict(torch.load(
    "{}.pt".format(config.cpt_load_path), 
    map_location=device)['net']
)
net.eval().half()

# data
dataloader = getDataLoader(config)[0]

In [31]:
with torch.no_grad():
    outputs = None
    frame = None
    for i, (frames, _) in enumerate(dataloader):
        # store subframe to a [100, *output.shape] tensor, i.e., outputs
        output = net(frames.half().to(device))
        if outputs == None: outputs = output
        else: outputs = torch.cat((outputs, output))

        # combine 100 subframe, i.e., outputs, to a frame
        if len(outputs) != 100: continue
        if frame == None: 
            frame  = dataloader.dataset.combineFrame(outputs) # type: ignore
        else:
            frame += dataloader.dataset.combineFrame(outputs) # type: ignore
        outputs = None
frame /= torch.max(frame)
imsave('data/eval/30.tif', (frame.cpu().detach() * 255).to(torch.uint8).numpy())